# Before we start, this is NOT the final competition we have planned for you at the end of the algocourse.

# Game Instructions:
Congrats on making it this far into the lecture! You'll be playing an auction game now, where there will be alternating rounds of buying and selling. 

#### Before the game:
Everyone in the room will **receive a unique integer value from 0 to 10, inclusive of 0 and 10**. 

The integer can be drawn from one of the following distributions with equal probability:
- Discerete uniform distributions: Uniform(0,5), Uniform(5,10), Uniform(2,7), Uniform(3,6)
- Binomial distributions: Binomial(n = 10, probability of success = 0.5), Binomial(n = 10, probability of success = 0.75)
- This probability density (where $C$ is a normalization constant):
\begin{equation*}
    f_{pdf}(x) = C*(\sin^2(50\pi x/10) + 0.01) * \exp(-50(x/10-0.5)^2), x\in[0,10]
\end{equation*}
- This probability density function (where $C$ is a normalization constant):
\begin{equation*}
    f_{pdf}(x) = C*(x/10)^{0.275}(\arctan{(x/10)} + 2^{x/10}) + \exp(1-21(x/10)), x\in[0,10]
\end{equation*}
- The distribution of the value of $Y_t=10*sigmoid(X_t)$ at time step $100$ of the following SDE (where sigmoid is the function $sigmoid(x)=\frac{1}{1+e^{-x}}$), $X_0=0$:
\begin{equation*}
dX_t = (50-X_t+20\sin(5X_t) + 10*1_{(X_t>80)})dt + (0.1 X_t (100 - X_t)\sin(10 X_t))dW_t
\end{equation*}

Note that for any continuous distributions, we draw a decimal value between 0 and 10 based on the pdf and then round to the nearest integer for the final value.

Some notation clarification: 
- $exp(x)=e^{x}$ the exponential function.
- $1_{X_t>80}$ is a function that equals $1$ if $X_t>80$ and $0$ otherwise.

**Hint:** In case it wasn't clear, it'll be hard figuring out EV.

#### During the game:
We will alternate between rounds of buying and selling in an auction-like fashion. **The item being auctioned is valued at the sum modulo 500 of all participants' drawn values**. For instance, if the sum of everyone's values is $1200$, then the value is $200$. Note that this means your bids should be anywhere from 0 to 499. Any value outside of that range will be assumed to be the nearest boundary (i.e. if its more than 499 it'll be taken as 499 and less than 0 is taken as 0).

How the rounds work: 
- During a **buying round**, everyone will get 10 seconds to place integer bids, which are prices they are willing to buy at. The people with the 10 highest bids receives 1 unit of the item.
    - Example: I place a bid at $2$, and 10 other people place bids at $4$,$5$ and $6$. Then in this round, I receive nothing while the other 10 people will get 1 unit of the item each at their respective bid prices.

- During a **selling round**, everyone will get 10 seconds to place integer asks, which are prices they are willing to sell at. The people with the 10 lowest asks gets to sell 1 unit of the item
    - Example: I place an ask at $10$ and 10 other people place asks at $9$, $8$ and $7$. Then in this round, I sell nothing while the other 10 people will sell 1 unit of the item each at their respective ask prices.
- The same price level, e.g. $4$, can have multiple people on it. If say there are 20 people who've placed a bid at $4$ and $4$ is the highest bid, then 10 people will be randomly selected from the 20.

How your wealth/money evolves:
- Whenever you successfully buy a unit, your wealth goes down by whatever price you set. E.g. if you set a bid at 10 and it was successful, then your wealth has reduced by 10.
- Whenever you successfully sell a unit, your wealth goes up by whatever price you set. E.g. if you set an ask at 10 and it was successful, then your wealth has increased by 10.

The goal of this game will be to develop an algorithm to buy low during the buy rounds and sell high during the sell rounds.

#### End of the game:
Whatever inventory is leftover at the end of the game will be evaluated at the actual sum of everyone's integers. For instance, if I bought during 3 of the buy rounds and sold in 10 of the sell rounds, then I am in total 7 units in the negative (i.e. short). If the sum value was $10$, then I have to subtract $7*10$ from the money/wealth I have now. If I was instead 7 units in the positive (i.e. long), then I have to add $7*10$ to the money/wealth I have now.

#### Prize:
If you have the most wealth by the end of this game AND no one else has the same wealth as you AND you won the bid/ask in at least HALF the rounds, **you get distinction on your cert by default**. If no one meets the above, no one gets the prize.

# Code
Each round consists of 10 units of time representing 10 seconds of total elapsed time, starting at time 1. Time 1 consists of the 1st second (0s to 1s), time 2 the 2nd second (1s to 2s), etc up until the 10th time unit which is the 10th second (9s to 10s). 

During each time unit, you can submit an order of what price you would like to bid/ask at, e.g. during time 1 you can choose to submit an order at price level 10 at 0.5 seconds. You can also choose to skip rounds entirely. These in between bids/asks are inconsequential, i.e. they are not what is executed. Time 10 is when you choose to make your final decision of where to place your bid/ask, if any at all. If you submitted an answer at some point during a round, the latest submission will be used in its place (if you fail to submit at a particular time unit). The market will then resolve (i.e. 10 people are chosen that round) using the values submitted at time 10. 

At the end of every time unit, i.e. at seconds 1,2,3...,9,10, you will receive snapshots of what everyone submitted for that round aggregated into an array of length 500 with indexing from 0 to 499. For example, if 10 people placed orders at price level 20, then ```array[20]``` will have $10$ in it. More generally, if $k$ people placed orders at price level $i$, then ```array[i]``` will have $k$ in it. Use this information to your advantage to infer what people believe the fair price to bid at will be.

You have 5 minutes to write out your logic. After this, the game will start. You can continue working on your strategy's logic as the game goes on, and simply rerun the appropriate cell below so that your orders are submitted according to your new strategy.

Note that in order to specify that you don't want to submit anything, simply submit ```None```. **If at any point you submit values outside of the 0 to 499 range**, it will be treated as a ```None``` submission and you will not participate.

In [1]:
# Insert your email here, the same one you used to fill out the attendance form. 
# Otherwise you will still be able to participate but won't get an integer and will have less info than everyone else.
# Note that this is what is used to update your wealth throughout the game, and 
# also what will appear on the leaderboard
EMAIL = "mongi.nouira24@imperial.ac.uk" # Switch this out with your actual email WITH the domain (i.e. if you have @imperial.ac.uk include it)

In [ ]:
class Strategy:
    def __init__(self):
        self.__my_number = None
        self.__snapshot = [0] * 100

    def __percentile_index(self, nonzero_indices, q):
        """Return approximate q-quantile index from sorted nonzero indices."""
        if not nonzero_indices:
            return None
        k = int((len(nonzero_indices) - 1) * q)
        return nonzero_indices[k]

    def buy_round(self, time_unit):
        nz = [i for i, x in enumerate(self.__snapshot) if x != 0]
        if not nz:
            return 250  # neutral start

        nz.sort()
        # top 10 / 500 ≈ 98th percentile
        buy_threshold = self.__percentile_index(nz, 0.98)

        # bid just below previous high zone
        value = max(0, min(buy_threshold + 2, 260))
    
        #print(f"buy {value}")
        return 40 #int(value)

    def sell_round(self, time_unit):
        nz = [i for i, x in enumerate(self.__snapshot) if x != 0]
        if not nz:
            return 300  # neutral start

        nz.sort()
        # bottom 10 / 500 ≈ 2nd percentile
        sell_threshold = self.__percentile_index(nz, 0.02)

        # ask just above previous low zone
        value = min(499, max(sell_threshold - 2, 240))
        
        #print(f"sell {value}")
        return 60 #int(value)

    def receive_int(self, num):
        self.__my_number = num

    def receive_snapshot(self, snapshot):
        self.__snapshot = snapshot

In [13]:
# Don't modify! Just run this cell if you want an active strategy running. Stop the cell and rerun if you want to run a new strategy.
# There are comments below in case your interested in how this section works
# Note that if you change any of the below, your submissions will simply be nulled so nothing will work

import nest_asyncio 
nest_asyncio.apply() # Lets asyncio work in ipython notebook cells by allowing nested loops.

import asyncio # Library to run asynchronous processes
import websockets # Lets you use websockets
import json # For serializing and deserializing json formatted data

async def run_strategy():
    uri = "ws://34.41.250.232:8765" # The websocket server address you will connect to
    strat = Strategy()
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps({
            "type" : "integer_request", 
            "email" : EMAIL 
        }))
        
        msg = await ws.recv()
        data = json.loads(msg)
        if data["num"] is not None:
            strat.receive_int(data["num"])
        
        while True:
            msg = await ws.recv() # Request from the server
            data = json.loads(msg) # Turns json msg into a python dictionary
            
            """
            # Contents of data

            data = {
                "round_number" : 1
                "round_type" : "buy",
                "time_unit" : 2,
                "prior_results" : [0,...,0],
                }
            
            # prior_results contains the 100 length array indexed from 0 to 99 that holds the aggregate of everyone's submissions
            # You will NOT be getting updates on your own personal wealth/whether your bid/ask worked or not.
            # This will all be displayed on a public leaderboard
            """
            
            strat.receive_snapshot(data["prior_results"]) # Update strat instance with the new auction array
            
            # If your code takes longer than ~1 second to generate the optimal bid_level/ask_level, 
            # Your submission will count towards the following time_unit
            # Should you take too long to submit at the 10th time_unit, you will be assumed to have no submission.
            if "FLAG" in data:
                break
            
            if data["round_type"] == "buy":
                bid_level = strat.buy_round(data["time_unit"])
                await ws.send(json.dumps({
                    "type" : "submission",
                    "email" : EMAIL,
                    "level" : bid_level,
                }))
                 
            elif data["round_type"] == "sell":
                ask_level = strat.sell_round(data["time_unit"])
                await ws.send(json.dumps({
                    "type" : "submission",
                    "email" : EMAIL,
                    "level" : ask_level,
                }))
            
asyncio.run(run_strategy())

buy 260
buy 260
buy 260
sell 240
sell 240
sell 240
sell 240
sell 240
sell 240
sell 240
sell 240
sell 240
sell 240
buy 260
buy 252
buy 252
buy 252
buy 252
buy 252
buy 252
buy 252


KeyboardInterrupt: 